In [4]:
import torch.nn as nn
import pandas as pd
import torch
import openslide
import sys
import imgaug.augmenters as iaa
sys.path.append('../brown-datathon/src')
from config import Config
from models.scse_pyramid_unet import UNet
import numpy as np

In [13]:
reader = openslide.OpenSlide('285317.svs')
whole_region = reader.read_region((0,0),1,reader.level_dimensions[1])
whole_region = np.array(whole_region.getdata()).reshape(*reader.level_dimensions[1][::-1],4)[:,:,:3]
patches = []
padder = iaa.PadToFixedSize(512,512,position='right-bottom',pad_cval=255)
for i in range(whole_region.shape[0] // 512 + 1):
    for j in range(whole_region.shape[1] // 512 + 1):
        patch = whole_region[i * 512: (i + 1) * 512,j * 512: (j + 1) * 512] 
        if i == whole_region.shape[0] // 512  or j == whole_region.shape[1] // 512 :
            patch = padder.augment_image(patch)
        patch = patch / 255
        patches.append(patch) 
input_patches = np.transpose(np.stack(patches),(0,3,1,2))

In [34]:
#config = Config()
models_list = ['no_shake_drop_no_deepcut','with_shake_drop_no_deepcut','no_shake_drop_deepcut','shake_drop_deepcut']
device = torch.device('cuda')
fprs = []
for i,model in enumerate(models_list):
    shake_drop = False if i % 2 == 0 else True
    model = UNet(84, 3, 3,
            shake_drop, True, 4, 2)
    model = model.to(device)
    checkpoint = torch.load('../brown-datathon/src/training_logs/' + model + '/best.pth.tar')
    model.load_state_dict(checkpoint.state_dict())
    preds = []
    with torch.no_grad():
        model.eval()
        for i in range(66):
            this_patches = torch.from_numpy(input_patches[i * 20:(i + 1) * 20]).float().to(device)
            output = model(this_patches)
            pred = torch.argmax(output,1).long().cpu().data.numpy()
            preds.append(pred)
            this_patches.to('cpu')
            del this_patches
    catted = np.concatenate(preds)
    recon_mask = np.vstack([np.hstack(catted[i * 42: (i + 1) * 42]) for i in range(31)])
    recon_mask = recon_mask[:reader.level_dimensions[1][1],:reader.level_dimensions[1][0]]
    fprs.append(len(recon_mask[recon_mask != 0]) / (np.prod(reader.level_dimensions[1])))

'\nwith torch.no_grad():\n    model.eval()\n    output = model(torch.from_numpy(t_region).float().to(device))\n    pred = torch.squeeze(torch.argmax(output,1).long()).cpu().data.numpy()\n    conts = []\n    for cl in range(2):\n        pred[pred != (cl + 1)] = 0\n        contours, hierarchy = cv2.findContours(pred.astype(np.uint8),cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)\n        conts.append(contours)\n        #y,x = np.mgrid[0:512,0:512]\n        #coord = np.vstack([x.ravel(),y.ravel()]).T\n        #print(x[pred == (cl + 1)])\n        #print(y[pred == (cl + 1)])\n        #print([np.min(pred[:, 1]) - 1, np.min(pred[:, 2]) - 1, np.max(pred[:, 1])+1,np.max(pred[:, 2]) + 1])\n'

In [9]:
bbox = pd.read_csv('285317.csv')
v_tau = bbox[bbox['label'] == 'vessel_tau']
np.flip(np.ceil(v_tau[['w','h']].values)).astype(np.uint32)
ves_region = reader.read_region(tuple(*np.floor(v_tau[['x','y']].values).astype(np.uint32)),0,tuple(*np.ceil(v_tau[['w','h']].values).astype(np.uint32)))
ves_region = np.array(ves_region.getdata()).reshape(*np.flip(*np.ceil(v_tau[['w','h']].values).astype(np.uint32)),4)[:,:,:3]
import matplotlib.pyplot as plt
plt.imshow(ves_region)